# LangChain `@tool` 데코레이터

In [1]:
from datetime import datetime
from dotenv import load_dotenv
import pytz
from langchain_core.tools import tool  # decorator
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [2]:
# .env 파일에 저장된 api_key를 OS 환경 변수로 로딩
load_dotenv()

True

In [3]:
# OpenAI 클라이언트를 생성
model = ChatOpenAI(model='gpt-4o-mini')

In [4]:
# AI에게 메시지 전달하고 실행(invoke).
messages = [
    SystemMessage(content='너는 사용자의 질문에 답하는 AI 비서야.'),
    HumanMessage(content='지금 현재 서울 시간?')
]
ai_message = model.invoke(input=messages)

In [5]:
type(ai_message)

langchain_core.messages.ai.AIMessage

In [6]:
print(ai_message)

content='현재 시간을 정확히 알 수는 없지만, 서울은 UTC+9 시간대에 위치해 있습니다. 당신이 현재 위치한 시간대에 따라 서울 시간을 계산할 수 있습니다. 예를 들어, 만약 현재 UTC 시간이 12시라면, 서울은 21시가 됩니다. 이러한 방법으로 현재 서울 시간을 확인할 수 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 30, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CFwclWypqEEKOJVTPIphzkNd7CRLD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--aa5488b4-7785-4de9-ad99-5b708bc8fc5c-0' usage_metadata={'input_tokens': 30, 'output_tokens': 72, 'total_tokens': 102, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [7]:
ai_message.pretty_print()

================================== Ai Message ==================================

현재 시간을 정확히 알 수는 없지만, 서울은 UTC+9 시간대에 위치해 있습니다. 당신이 현재 위치한 시간대에 따라 서울 시간을 계산할 수 있습니다. 예를 들어, 만약 현재 UTC 시간이 12시라면, 서울은 21시가 됩니다. 이러한 방법으로 현재 서울 시간을 확인할 수 있습니다.


도구(tool)을 AI에게 제공하고, AI는 tool 목록에 있는 함수 호출(function calling)을 요청해서 에이전트가 함수 호출 결과를 다시 AI에게 전송하면 AI는 함수 호출 결과를 바탕으로 답변을 생성할 수 있음.

사용자 질문(도구 목록 제공) -> AI 도구 호출 요청 -> 사용자 함수 호출 결과 -> AI 답변 생성.

In [8]:
@tool
def get_current_time(timezone: str, location: str) -> str:
    """해당 timezone의 현재 날짜와 시간을 문자열로 리턴.

    Args:
        timezone (str): 타임존. (예: Asia/Seoul)
        location (str): 지역명. 타임존은 모든 도시 이름에 대응되지 않기 때문에 LLM이 답변을 생성할 때 이용하도록 제공.
    Returns: '날짜 시간 타임존(지역명)' 형식의 문자열을 반환. (예: 2025-09-15 15:30:55 Asia/Seoul(부산))
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')
    result = f'{now} {timezone}({location})'

    return result

In [9]:
# 도구 목록: @tool 데코레이터가 사용된 함수들의 리스트.
tools = [get_current_time,]

In [10]:
# AI에서 도구 목록의 함수 호출을 요청했을 때 함수 객체를 쉽게 찾기 위해서 dict 선언.
# 함수 이름을 key로, 함수 객체를 value로 갖는 딕셔너리.
tool_dict = {
    'get_current_time': get_current_time,
}